In [17]:
import re
import qiskit
import json
import time
import numpy as np

from qiskit import transpile
from qiskit_aer import AerSimulator
from qiskit.converters import circuit_to_dag

from fast_generator import fc_tree_commute_recur_lookahead_fast
from absorption import extract_CNOT_network, update_probabilities

from benchmarks.UCCSD_entanglers import generate_UCCSD_entanglers
# from circuit_generator import construct_qcc_circuit
from utilities import generate_pauli_strings
from circuit_generator import generate_opt_circuit, construct_qcc_circuit

In [18]:
#QAOA for MAXCUT 
cost_hamiltonian = ['IIIZZ', 'IIZIZ', 'ZIIIZ', 'IIZZI', 'IZZII', 'ZZIII']
mixer_hamiltonian = ['XIIII', 'IXIII', 'IIXII', 'IIIXI', 'IIIIX']
test_paulis = (cost_hamiltonian + mixer_hamiltonian)* 2
test_params = [3.271] * 6 +   [2.874] * 5 + [2.641] * 6 + [0.9339]* 5

start_time = time.time()
opt_qc, append_clifford, sorted_entanglers = fc_tree_commute_recur_lookahead_fast(entanglers=test_paulis, params=test_params, barrier=False)
opt_qiskit = transpile(opt_qc, optimization_level=3, basis_gates=["cx", "sx", "x", "rz"])

#need to add a layer of hadamard gates:
for i in range(0, len(test_paulis[0])):
    opt_qc.h(i)
opt_qc.measure_active()
end_time = time.time()
combined_time = end_time - start_time


origin_qc = construct_qcc_circuit(entanglers = test_paulis, params = test_params, barrier=False)
#generate the qiskit optimized circuit
origin_qiskit = transpile(origin_qc, optimization_level = 3, basis_gates = ["cx", "sx", "x", "rz"])
origin_qiskit.measure_active() 
origin_qiskit.count_ops()['cx']

24

In [19]:
append_clifford.draw()

┌───┐                              ┌───┐     ┌───┐     »
q_0: ────────────■──┤ H ├─────────────────■────────────┤ X ├──■──┤ H ├─────»
               ┌─┴─┐├───┤┌───┐            │  ┌───┐┌───┐└─┬─┘  │  └───┘     »
q_1: ───────■──┤ X ├┤ X ├┤ H ├───────■────┼──┤ X ├┤ H ├──┼────┼────────────»
          ┌─┴─┐├───┤└─┬─┘└───┘       │    │  └─┬─┘└───┘  │  ┌─┴─┐┌───┐     »
q_2: ──■──┤ X ├┤ H ├──┼──────────────┼────┼────■─────────┼──┤ X ├┤ H ├──■──»
       │  └───┘└───┘  │  ┌───┐┌───┐  │    │       ┌───┐  │  └───┘├───┤┌─┴─┐»
q_3: ──┼──────────────┼──┤ X ├┤ H ├──┼────┼────■──┤ H ├──┼───────┤ X ├┤ X ├»
     ┌─┴─┐            │  └─┬─┘├───┤┌─┴─┐┌─┴─┐┌─┴─┐└───┘  │  ┌───┐└─┬─┘├───┤»
q_4: ┤ X ├────────────■────■──┤ H ├┤ X ├┤ X ├┤ X ├───────■──┤ H ├──■──┤ H ├»
     └───┘                    └───┘└───┘└───┘└───┘          └───┘     └───┘»
«     ┌───┐┌───┐                                   
«q_0: ┤ X ├┤ H ├────────────■─────────■────■────■──
«     └─┬─┘├───┤┌───┐       │  ┌───┐  │    │  ┌─┴─┐
«q_1: ──┼──┤ X ├┤ H ├───────┼──┤ X ├──┼────┼──┤ X ├
«       │  └─┬─┘├───┤┌───┐  │  └─┬─┘  │  ┌─┴─┐└───┘
«q_2: ──┼────■──┤ H ├┤ X ├──┼────■────┼──┤ X ├─────
«       │  ┌───┐└───┘└─┬─┘┌─┴─┐       │  └───┘     
«q_3: ──■──┤ H ├──■────■──┤ X ├───────┼────────────
«          └───┘┌─┴─┐     └───┘     ┌─┴─┐          
«q_4: ──────────┤ X ├───────────────┤ X ├──────────
«               └───┘               └───┘

In [20]:
opt_qc.count_ops()

OrderedDict([('cx', 22),
             ('rz', 22),
             ('h', 20),
             ('measure', 5),
             ('barrier', 1)])

In [21]:
opt_qc.draw()

»
      q_0: ──■─────────────────■─────────────────■─────────────────■──»
           ┌─┴─┐┌───────────┐  │                 │      ┌───┐      │  »
      q_1: ┤ X ├┤ Rz(3.271) ├──┼─────────────────┼──────┤ X ├──────┼──»
           └───┘└───────────┘┌─┴─┐┌───────────┐  │      └─┬─┘      │  »
      q_2: ──────────────────┤ X ├┤ Rz(3.271) ├──┼────────■────────┼──»
                             └───┘└───────────┘  │               ┌─┴─┐»
      q_3: ──────────────────────────────────────┼───────────────┤ X ├»
                                               ┌─┴─┐┌───────────┐└───┘»
      q_4: ────────────────────────────────────┤ X ├┤ Rz(3.271) ├─────»
                                               └───┘└───────────┘     »
measure: 5/═══════════════════════════════════════════════════════════»
                                                                      »
«               ┌───┐                                       ┌───┐    »
«      q_0: ────┤ H ├───────────────────────────────────────┤ X ├────»
«           ┌───┴───┴───┐    ┌───┐    ┌───────────┐┌───┐    └─┬─┘    »
«      q_1: ┤ Rz(3.271) ├────┤ H ├────┤ Rz(2.874) ├┤ X ├──────┼──────»
«           └───┬───┬───┘┌───┴───┴───┐└───┬───┬───┘└─┬─┘      │      »
«      q_2: ────┤ X ├────┤ Rz(3.271) ├────┤ H ├──────■────────┼──────»
«               └─┬─┘    └───────────┘    ├───┤               │      »
«      q_3: ──────■────────────■──────────┤ H ├───────────────■──────»
«                            ┌─┴─┐    ┌───┴───┴───┐┌───┐┌───────────┐»
«      q_4: ─────────────────┤ X ├────┤ Rz(3.271) ├┤ H ├┤ Rz(2.874) ├»
«                            └───┘    └───────────┘└───┘└───────────┘»
«measure: 5/═════════════════════════════════════════════════════════»
«                                                                    »
«           ┌───────────┐┌───┐                               ┌───┐┌───────────┐»
«      q_0: ┤ Rz(2.874) ├┤ H ├───────────────■───────────────┤ X ├┤ Rz(2.641) ├»
«           ├───────────┤├───┤┌───────────┐  │               └─┬─┘└───┬───┬───┘»
«      q_1: ┤ Rz(2.874) ├┤ H ├┤ Rz(2.641) ├──┼─────────────────┼──────┤ X ├────»
«           └───────────┘├───┤└───────────┘┌─┴─┐┌───────────┐  │      └─┬─┘    »
«      q_2: ──────■──────┤ H ├─────────────┤ X ├┤ Rz(2.641) ├──┼────────■──────»
«               ┌─┴─┐    ├───┤┌───────────┐├───┤└───────────┘  │               »
«      q_3: ────┤ X ├────┤ X ├┤ Rz(2.874) ├┤ H ├───────────────┼────────■──────»
«               └───┘    └─┬─┘└───┬───┬───┘└───┘               │      ┌─┴─┐    »
«      q_4: ───────────────■──────┤ H ├────────────────────────■──────┤ X ├────»
«                                 └───┘                               └───┘    »
«measure: 5/═══════════════════════════════════════════════════════════════════»
«                                                                              »
«                                 ┌───┐     ┌────────────┐                  »
«      q_0: ───────────────■──────┤ H ├─────┤ Rz(0.9339) ├──────────────────»
«           ┌───────────┐  │      └───┘     └────────────┘    ┌───┐         »
«      q_1: ┤ Rz(2.641) ├──┼──────────────────────■───────────┤ H ├─────────»
«           └───┬───┬───┘  │  ┌────────────┐      │           └───┘         »
«      q_2: ────┤ H ├──────┼──┤ Rz(0.9339) ├──────┼─────────────────────────»
«               ├───┤      │  └────────────┘      │                         »
«      q_3: ────┤ H ├──────┼──────────────────────┼─────────────────────────»
«           ┌───┴───┴───┐┌─┴─┐                  ┌─┴─┐     ┌───────────┐┌───┐»
«      q_4: ┤ Rz(2.641) ├┤ X ├──────────────────┤ X ├─────┤ Rz(2.641) ├┤ H ├»
«           └───────────┘└───┘                  └───┘     └───────────┘└───┘»
«measure: 5/════════════════════════════════════════════════════════════════»
«                                                                           »
«                                                      ┌───┐                   »
«      q_0: ────────────────────────────────────────■──┤ H ├───

In [22]:
cnot_network = extract_CNOT_network(append_clifford)

In [23]:
#Run the original circuit on a noiseless simulator:
simulator = AerSimulator()
origin_result = simulator.run(origin_qiskit, shots = 10000).result()

In [24]:
def get_top_k_values(input_dict, k):
    # Sort the dictionary by values in descending order and return the top k items
    sorted_items = sorted(input_dict.items(), key=lambda item: item[1], reverse=True)
    return sorted_items[:k]

In [25]:
origin_probs= origin_result.get_counts()

In [26]:
top_k_states_origin = get_top_k_values(origin_probs, 20)
print(top_k_states_origin)

[('11111', 3622), ('11011', 1051), ('11110', 1022), ('01111', 679), ('11101', 663), ('10111', 657), ('11010', 401), ('10110', 209), ('01011', 194), ('11100', 177), ('01110', 176), ('11001', 175), ('01101', 140), ('10011', 138), ('00111', 127), ('10101', 121), ('10010', 66), ('01010', 62), ('11000', 50), ('00110', 43)]


In [27]:
#Run the optimized circuit on a noiseless simulator:

opt_result = simulator.run(opt_qc, shots = 10000).result()

In [28]:
dag = circuit_to_dag(cnot_network)
updated_states = {}

updated_probs = update_probabilities(opt_result.get_counts(), dag)


In [29]:
from qiskit.quantum_info.analysis import hellinger_fidelity

In [30]:
hellinger_fidelity(origin_probs, updated_probs)

0.9983897185239435

In [31]:
top_k_states_origin = get_top_k_values(origin_probs, 20)
print(top_k_states_origin)

[('11111', 3622), ('11011', 1051), ('11110', 1022), ('01111', 679), ('11101', 663), ('10111', 657), ('11010', 401), ('10110', 209), ('01011', 194), ('11100', 177), ('01110', 176), ('11001', 175), ('01101', 140), ('10011', 138), ('00111', 127), ('10101', 121), ('10010', 66), ('01010', 62), ('11000', 50), ('00110', 43)]


In [32]:
top_k_states_updated = get_top_k_values(updated_probs, 20)
print(top_k_states_updated)

[('11111', 3616), ('11110', 1067), ('11011', 1054), ('11101', 686), ('01111', 668), ('10111', 666), ('11010', 368), ('01011', 220), ('10110', 198), ('11001', 180), ('11100', 179), ('10011', 153), ('01110', 152), ('00111', 138), ('01101', 121), ('10101', 109), ('01010', 64), ('11000', 48), ('10010', 46), ('00011', 45)]
